In [13]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json

# Load API key from .env file
load_dotenv()
coinapi_key = os.getenv('api_key')

exchange_id = "DERIBIT"

# CoinAPI endpoint for symbols metadata
url = f"https://rest.coinapi.io/v1/options/{exchange_id}/current"

#payload = {}
headers = {
  'Accept': 'text/plain',
  'X-CoinAPI-Key': coinapi_key
}

#response = requests.request("GET", url, headers=headers, data=payload)
response = requests.request("GET", url, headers=headers)

#print(response.text)

# Check for successful response
if response.status_code == 200:
    data = response.json()
    
    # Filter for BTC/USD call options
    filtered_calls = []
    for option in data:
        if option.get("asset_id_base") == "BTC" and option.get("asset_id_quote") == "USD":
            underlying_price = option.get("underlying_price")
            time_expiration = option.get("time_expiration")

            for strike in option.get("strikes", []):
                if "call" in strike:
                    call_data = {
                        "symbol_id": strike["call"]["symbol_id"],
                        "time_exchange": strike["call"]["time_exchange"],
                        "time_coinapi": strike["call"]["time_coinapi"],
                        "strike_price": strike["strike_price"],
                        "ask_price": strike["call"]["ask_price"],
                        "ask_size": strike["call"]["ask_size"],
                        "bid_price": strike["call"]["bid_price"],
                        "bid_size": strike["call"]["bid_size"],
                        "underlying_price": underlying_price,
                        "time_expiration": time_expiration
                    }
                    filtered_calls.append(call_data)

    # Convert to DataFrame for better visualization
    df_calls = pd.DataFrame(filtered_calls)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', None)
    print(df_calls)

else:
    print(f"Error: {response.status_code}, {response.text}")


                               symbol_id                 time_exchange                  time_coinapi  strike_price  ask_price  ask_size  bid_price  bid_size  underlying_price               time_expiration
0    DERIBIT_OPT_BTC_USD_250425_104000_C  2025-02-03T17:20:35.4024035Z  2025-02-03T17:20:35.4024302Z      104000.0     0.0950      22.1     0.0935       1.3      98717.420348  2025-04-25T08:00:00.0000000Z
1    DERIBIT_OPT_BTC_USD_250425_170000_C  2025-02-03T17:20:28.2097876Z  2025-02-03T17:20:28.2098043Z      170000.0     0.0090      28.1     0.0080      52.9      98717.420348  2025-04-25T08:00:00.0000000Z
2    DERIBIT_OPT_BTC_USD_250425_116000_C  2025-02-03T17:20:29.7027605Z  2025-02-03T17:20:29.7027779Z      116000.0     0.0580       1.4     0.0570      15.3      98717.420348  2025-04-25T08:00:00.0000000Z
3    DERIBIT_OPT_BTC_USD_250425_175000_C  2025-02-03T17:20:22.4903184Z  2025-02-03T17:20:22.4903292Z      175000.0     0.0080      46.7     0.0070      21.8      98717.420348  2025